In [ ]:
import pandas as pd

#df = pd.read_pickle('./dataset/State_Hotel_reviews_v1.pkl')
#df = pd.read_csv('./State_Hotel_reviews_v1_pred.txt', encoding='utf-8', sep='\t')

In [ ]:
df.head(3)

In [ ]:
df.columns

In [ ]:
print(f'How many records: {len(df)}')

In [ ]:
print(f'Distribution of [Overall Rating]:\n{df["Overall Rating"].value_counts(normalize=True)}')
print(f'Distribution of perspective [Value]:\n{df["Value"].value_counts(normalize=True)}')
print(f'Distribution of perspective [Location]:\n{df["Location"].value_counts(normalize=True)}')
print(f'Distribution of perspective [Rooms]:\n{df["Rooms"].value_counts(normalize=True)}')
print(f'Distribution of perspective [Service]:\n{df["Service"].value_counts(normalize=True)}')
print(f'Distribution of perspective [Sleep Quality]:\n{df["Sleep Quality"].value_counts(normalize=True)}')
print(f'Distribution of perspective [Cleanliness]:\n{df["Cleanliness"].value_counts(normalize=True)}')

In [ ]:
def get_lemmatize_tokens(text, wo_stopword=True):
    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import TweetTokenizer
    import string
    from nltk.stem import WordNetLemmatizer
    import re

    text = " ".join([re.sub(r'^h?ttps?:\/\/.*[\r\n]*', '', item, flags=re.MULTILINE) for item
                     in text.split()])
                     #in text.replace('\n', ' ').strip().replace('\xa0', '').split()])

    text = " ".join([re.sub(r'^doi:.*[\r\n]*', '', item, flags=re.MULTILINE) for item
                     in text.split()])

    text = " ".join([re.sub(r'^www.*[\r\n]*', '', item, flags=re.MULTILINE) for item
                     in text.split()])

    lemmatizer = WordNetLemmatizer()
    string_punctuation=list(f'{string.punctuation}:”“….....’–‘—•')
    #with open("./dataset/stopwords_manual.csv", "r", encoding='utf-8') as f:
    #    stopwords_manual = [item for item in f.read().split("\n") if item]
    sp = set(string_punctuation +
             stopwords.words('english') +
             #stopwords_manual +
             ['\xa0', '\xad', 'et al', '\uf07c', '\x00'])

    tokens = [token for token in nltk.TweetTokenizer().tokenize(text.strip().lower())]

    if wo_stopword:
        tokens = [token for token in tokens if token not in sp
                  #and token not in stopwords_manual
                 ]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

In [ ]:
%%time

df["text"] = df["Review Title"].astype(str) + ' ' + df["Review Content"].astype(str)
#df["token"] = df['text'].replace("\xa0", " ").replace("\n", " ")
df["token"] = df['text'].map(get_lemmatize_tokens)

In [ ]:
df.head(3)

In [ ]:
df["Travel Type Label"] = df["Travel Type"].map({"Traveled on business":"Business/Solo",
                                                 "business":"Business/Solo",
                                                 "Traveled solo":"Business/Solo",
                                                 "Traveled as a couple":"Friend/Couple",
                                                 "friends":"Friend/Couple",
                                                 "Traveled with friends":"Friend/Couple",
                                                 "Traveled with family":"Family",
                                                 "family":"Family",
                                                })

df["Travel Type Label"].value_counts()

# Load Dataset Pickle file

In [ ]:
import pandas as pd

df = pd.read_pickle('./dataset/State_Hotel_reviews_v1_240415.01.pkl')

# Topic Modeling

In [ ]:
version='240418.01'

## Business/Solo Positive (4 or 5 顆星)

Location, Service, Sleep Quality, Cleanliness

In [ ]:
#Location, Service, Sleep Quality, Cleanliness
travel_type = "Business/Solo"#"Friend/Couple", "Family", "Business/Solo"
df_business_pos = df[df['Travel Type Label'].str.contains(travel_type, na=False, regex=False)]
df_business_pos = df_business_pos [#df_business_pos ["Overall Rating"].isin([4, 5]) &
           #df_business_pos ["Value"].isin([4, 5]) &
           df_business_pos ["Location"].isin([4, 5]) &
           #df_business_pos ["Rooms"].isin([4, 5]) &
           df_business_pos ["Service"].isin([4, 5]) &
           df_business_pos ["Sleep Quality"].isin([4, 5]) &
           df_business_pos ["Cleanliness"].isin([4, 5])]

print(f'{travel_type} positive: {len(df_business_pos)}')

In [ ]:
df_sample = df_business_pos

In [ ]:
docs = df_sample['token'].map(" ".join).tolist()
docs[:3]

In [ ]:
%%time

import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["TOKENIZERS_PARALLELISM"]="true"

from umap import UMAP
from hdbscan import HDBSCAN
#from cuml.cluster import HDBSCAN
#from cuml.manifold import UMAP
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer


seed = 42

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=30, #20,#15,
                  n_components=2,
                  min_dist=0.05,
                  metric='cosine',
                  random_state=seed)

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=300,
                        metric='euclidean',
                        cluster_selection_method='eom',
                        prediction_data=True)

# Step 4 - Tokenize topics
#def tokenizer_split(text):
#    return text.split(',')

vectorizer_model = CountVectorizer(ngram_range=(2, 3), stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# Step 6 - (Optional) Fine-tune topic representations with
# a `bertopic.representation` model
representation_model = KeyBERTInspired(top_n_words=100)

# All steps together
topic_model = BERTopic(
    embedding_model=embedding_model,          # Step 1 - Extract embeddings
    umap_model=umap_model,                    # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
    representation_model=representation_model, # Step 6 - (Optional) Fine-tune topic represenations
    top_n_words=100,
    #min_topic_size=100,
)

topics, probs = topic_model.fit_transform(docs)
#topic_model.fit_transform(docs)
#topic_model.reduce_topics(docs, nr_topics='auto') 

In [ ]:
topic_model.save(f"./output/model_{travel_type.replace('/','-')}_pos_{version}")

In [ ]:
topic_model.get_topic_info()

In [ ]:
%%time

topic_model.visualize_topics()


In [ ]:
%%time

topic_model.visualize_documents(docs)


In [ ]:
%%time

# 濾掉 topic==-1 的 documents
valid_docs_index = [idx for idx, topic in enumerate(topics) if topic != -1]
filtered_docs = [docs[idx] for idx in valid_docs_index]
filtered_topics = [topics[idx] for idx in valid_docs_index]

topic_model.visualize_documents(docs=filtered_docs, topics=filtered_topics, embeddings=embedding_model)

In [ ]:
%%time

topic_model.visualize_barchart(topics=list(set(topics))[:10], n_words=30)


In [ ]:
topic_model.visualize_heatmap()


In [ ]:
%%time

reduced_topics, reduced_probs = topic_model.reduce_topics(docs, nr_topics='auto')
#topic_model.reduce_topics(docs, nr_topics=12) #auto

In [ ]:
topic_model.get_topic_info()


In [ ]:
%%time

topic_model.visualize_topics()


In [ ]:
topic_model.visualize_documents(docs)

In [ ]:
pd.DataFrame([topic_model.get_topic(i) for i in range(len(topic_model.topic_sizes_)-1)]).transpose()

In [ ]:
df_topickeywords = pd.DataFrame([topic_model.get_topic(i)
                                 for i in range(len(topic_model.topic_sizes_)-1)]).transpose()

df_topickeywords.to_pickle(f"./output/df_keywords_{travel_type.replace('/','-')}_pos_{version}.pkl")
df_topickeywords.to_csv(f"./output/keywords_{travel_type.replace('/','-')}_pos_{version}.csv",
                        encoding='utf-8', index=False)

df_topickeywords

In [ ]:
for i in range(len(topic_model.topic_sizes_)-1):
    print(f'### Topic {i}:')
    print(f'```python!')
    print(f'{[item[0] for item in topic_model.get_topic(i)[:30]]}')
    print(f'```')